In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_parquet("./data/processed/deepchem_mol2vec_300.parquet")
labels = df["exp"].to_numpy()
features = df.drop(["exp", "smiles", "CMPD_CHEMBLID"], axis = 1)
features = features.to_numpy()
labels = labels.reshape(-1, 1)
features = np.stack(features.squeeze())

morgan_df = pd.read_parquet("./data/processed/deepchem_morgan_fp.parquet")
morgan_features = morgan_df.drop(["exp"], axis = 1)
morgan_features = morgan_features.to_numpy()
morgan_features = np.stack(morgan_features.squeeze())

extended_df = pd.read_parquet("./data/processed/deepchem_extended_mol2vec_300.parquet")
extended_features = extended_df.drop(["exp", "smiles", "CMPD_CHEMBLID"], axis=1)

mol2vec_col = extended_df["mol2vec"]
extended_descriptors = [
    "molwt", "clogp", "hba", "hbd",
    "tpsa",
    "num_rotatable_bonds",
    "num_rings",
    "num_aromatic_rings",
    "fraction_csp3",
    "num_heavy_atoms",
    "num_valence_electrons"
]


features_mol2vec = np.array(mol2vec_col.tolist(), dtype=np.float64)
features_descriptors = extended_df[extended_descriptors].to_numpy(dtype=np.float64)

extended_features = np.hstack((features_mol2vec, features_descriptors))

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import clone
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer 

def run_stratified_cv(
    pipeline: Pipeline,
    features: np.ndarray,
    labels: np.ndarray,
    n_splits: int = 5,
    n_bins: int = 10,
    random_state: int = 42,
    transform_target: bool = False,
    verbose: bool = True
) -> tuple[np.ndarray, np.ndarray, float, float, float, float]:
    if labels.ndim > 1:
      labels = labels.ravel() 

    logp_bins = pd.cut(labels, bins=n_bins, labels=False, duplicates="drop")
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    offset = 0.0 
    if transform_target:
        min_label = np.min(labels)
        if min_label <= 0:
            offset = abs(min_label) + 1 
        else:
            offset = 1 
            
    all_rmse, all_r2, all_y_pred, all_y_test = [], [], [], []
    if verbose:
        print(f"\nStarting {n_splits}-Fold CV for:\n{pipeline}")

    for fold, (train_idx, test_idx) in enumerate(skf.split(features, logp_bins), 1):
        X_train, X_test = features[train_idx], features[test_idx]
        y_train_orig, y_test_orig = labels[train_idx], labels[test_idx] 

        fold_pipeline = clone(pipeline)
        if transform_target:
            y_train_transformed = np.log(y_train_orig + offset)
            target = y_train_transformed
        else:
            target = y_train_orig

        fold_pipeline.fit(X_train, target)
        y_pred_raw = fold_pipeline.predict(X_test) 
        if transform_target:
            y_pred_original_scale = np.exp(y_pred_raw) - offset
        else:
            y_pred_original_scale = y_pred_raw

        all_y_pred.append(y_pred_original_scale)
        all_y_test.append(y_test_orig)
        rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_original_scale))
        r2 = r2_score(y_test_orig, y_pred_original_scale)
        if verbose:
            print(f"Fold {fold} RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")

        all_rmse.append(rmse)
        all_r2.append(r2)

    final_y_pred = np.concatenate(all_y_pred)
    final_y_test = np.concatenate(all_y_test)
    mean_rmse = np.nanmean(all_rmse) 
    std_rmse = np.nanstd(all_rmse)  
    mean_r2 = np.nanmean(all_r2)
    std_r2 = np.nanstd(all_r2)

    print(f"\nSummary for: {pipeline}")
    print(f"Average RMSE: {mean_rmse:.4f} +/- {std_rmse:.4f}")
    print(f"Average R2 Score: {mean_r2:.4f} +/- {std_r2:.4f}")
    print("-------------------------------------------------") 

    return final_y_pred, final_y_test, mean_rmse, std_rmse, mean_r2, std_r2


In [ ]:
print("\nRunning SVR Pipelines:")

pipeline_svr_robust = make_pipeline(RobustScaler(), SVR(C=7.42, epsilon=0.05, gamma="scale", kernel="rbf"))
svr_results = {}
                                    

print("\nMol2Vec")
svr_results["mol2vec"] = run_stratified_cv(
    pipeline=pipeline_svr_robust,
    features=features,
    labels=labels,
    transform_target=False
)

print("\nMol2Vec Extended")
svr_results["extended_features"] = run_stratified_cv(
    pipeline=pipeline_svr_robust,
    features=extended_features,
    labels=labels,
    transform_target=False
)
print("\nMorgan Fingerprint")
svr_results["morgan"] = run_stratified_cv(
    pipeline=pipeline_svr_robust,
    features=morgan_features,
    labels=labels,
    transform_target=False
)


In [ ]:
from lightgbm import LGBMRegressor
import warnings


warnings.filterwarnings(
    "ignore",
    message=r"X does not have valid feature names, but LGBMRegressor was fitted with feature names",
    category=UserWarning
)

print("\nRunning LightGBM Pipelines:")
best_max_depth = 9
best_n_estimators = 2300
best_learning_rate = 0.028071814963637386
best_num_leaves = 21
best_feature_fraction = 0.6976128568411865
best_bagging_fraction = 0.5513242454920146
best_bagging_freq = 4
best_min_child_samples = 18
best_reg_alpha = 0.0001570177895423675
best_reg_lambda = 1.1649586097255062e-05

lgbm_sklearn = LGBMRegressor(
    boosting_type="gbdt",        
    objective="regression",      
    max_depth=best_max_depth,
    num_leaves=best_num_leaves,
    learning_rate=best_learning_rate,
    n_estimators=best_n_estimators,
    feature_fraction=best_feature_fraction,
    bagging_fraction=best_bagging_fraction, 
    bagging_freq=best_bagging_freq,         
    min_child_samples=best_min_child_samples,
    reg_alpha=best_reg_alpha,
    reg_lambda=best_reg_lambda,
    n_jobs=-1,                  
    random_state=42,         
    verbose=-1                   
)

pipeline_lgbm_noscaling = make_pipeline(
    lgbm_sklearn
)
lgbm_results = {}



print("\nMol2Vec")
lgbm_results["mol2vec"] = run_stratified_cv(
    pipeline=pipeline_lgbm_noscaling,
    features=features, 
    labels=labels,
    transform_target=False,
    verbose=False
)

print("\nMol2Vec Extended")
lgbm_results["extended_features"] = run_stratified_cv(
    pipeline=pipeline_lgbm_noscaling,
    features=extended_features, 
    labels=labels,
    transform_target=False,
    verbose=False
)

print("\nMorgan Fingerprint")
lgbm_results["morgan"] = run_stratified_cv(
    pipeline=pipeline_lgbm_noscaling,
    features=morgan_features, 
    labels=labels,
    transform_target=False,
    verbose=False
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

feature_sets = list(svr_results.keys())
svr_rmse = [svr_results[fs][2] for fs in feature_sets]
lgbm_rmse = [lgbm_results[fs][2] for fs in feature_sets]
svr_std_rmse = [svr_results[fs][3] for fs in feature_sets]
lgbm_std_rmse = [lgbm_results[fs][3] for fs in feature_sets]

svr_r2 = [svr_results[fs][4] for fs in feature_sets]
lgbm_r2 = [lgbm_results[fs][4] for fs in feature_sets]
svr_std_r2 = [svr_results[fs][5] for fs in feature_sets]
lgbm_std_r2 = [lgbm_results[fs][5] for fs in feature_sets]

x = np.arange(len(feature_sets))
width = 0.35

fig_rmse, ax_rmse = plt.subplots(figsize=(8, 6))
rects1_rmse = ax_rmse.bar(x - width/2, svr_rmse, width, label="SVR", yerr=svr_std_rmse, capsize=5, color="tomato")
rects2_rmse = ax_rmse.bar(x + width/2, lgbm_rmse, width, label="LightGBM", yerr=lgbm_std_rmse, capsize=5, color="skyblue")

ax_rmse.set_ylabel("Mean RMSE", fontsize=12)
ax_rmse.set_title("Mean RMSE ($\pm$ Std Dev)", fontsize=14)
ax_rmse.set_xticks(x)
ax_rmse.set_xticklabels(feature_sets, rotation=45, ha="right", fontsize=10)
ax_rmse.tick_params(axis="y", labelsize=10)
ax_rmse.legend(title="Model", fontsize=10, title_fontsize=11)
ax_rmse.bar_label(rects1_rmse, padding=3, fmt="%.3f", fontsize=9)
ax_rmse.bar_label(rects2_rmse, padding=3, fmt="%.3f", fontsize=9)
max_rmse_val = max(max(svr_rmse), max(lgbm_rmse))
max_err_rmse = max(max(svr_std_rmse), max(lgbm_std_rmse))
ax_rmse.set_ylim(bottom=0, top=max_rmse_val + max_err_rmse + 0.05)
fig_rmse.tight_layout()
plt.savefig("rmse_bar.png")


fig_r2, ax_r2 = plt.subplots(figsize=(8, 6))
rects1_r2 = ax_r2.bar(x - width/2, svr_r2, width, label="SVR", yerr=svr_std_r2, capsize=5, color="tomato")
rects2_r2 = ax_r2.bar(x + width/2, lgbm_r2, width, label="LightGBM", yerr=lgbm_std_r2, capsize=5, color="skyblue")

ax_r2.set_ylabel("Mean R2 Score", fontsize=12)
ax_r2.set_title("Mean R2 Score ($\pm$ Std Dev)", fontsize=14)
ax_r2.set_xticks(x)
ax_r2.set_xticklabels(feature_sets, rotation=45, ha="right", fontsize=10)
ax_r2.tick_params(axis="y", labelsize=10)
ax_r2.legend(title="Model", fontsize=10, title_fontsize=11)
ax_r2.bar_label(rects1_r2, padding=3, fmt="%.3f", fontsize=9)
ax_r2.bar_label(rects2_r2, padding=3, fmt="%.3f", fontsize=9)
max_r2_val = max(max(svr_r2), max(lgbm_r2))
max_err_r2 = max(max(svr_std_r2), max(lgbm_std_r2))
ax_r2.set_ylim(bottom=0, top=max_r2_val + max_err_r2 + 0.05)
fig_r2.tight_layout()
plt.savefig("r2_bar.png")

###

feature_set_to_plot = "extended_features"
svr_data = svr_results[feature_set_to_plot]
svr_actual = svr_data[1]
svr_predicted = svr_data[0]
svr_rmse_scatter = svr_data[2] 
svr_model_name = f"SVR - extended features"

plt.figure(figsize=(7, 7))
plt.scatter(svr_actual, svr_predicted, alpha=0.5, label=f"RMSE = {svr_rmse_scatter:.4f}", color="steelblue")
min_val_svr = min(min(svr_actual), min(svr_predicted))
max_val_svr = max(max(svr_actual), max(svr_predicted))
plt.plot([min_val_svr, max_val_svr], [min_val_svr, max_val_svr], "r--", label="y=x")
plt.title(f"Predicted vs. Actual LogP\n{svr_model_name}", fontsize=14)
plt.xlabel("Actual LogP", fontsize=12)
plt.ylabel("Predicted LogP", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=10)
plt.axis("equal")
plt.tight_layout()
plt.savefig("scatter_svr.png")

lgbm_data = lgbm_results["extended_features"]
lgbm_actual = lgbm_data[1]
lgbm_predicted = lgbm_data[0]
lgbm_rmse_scatter = lgbm_data[2] 
lgbm_model_name = f"LightGBM - extended features"

plt.figure(figsize=(7, 7))
plt.scatter(lgbm_actual, lgbm_predicted, alpha=0.5, label=f"RMSE = {lgbm_rmse_scatter:.4f}", color="steelblue")
min_val_lgbm = min(min(lgbm_actual), min(lgbm_predicted))
max_val_lgbm = max(max(lgbm_actual), max(lgbm_predicted))
plt.plot([min_val_lgbm, max_val_lgbm], [min_val_lgbm, max_val_lgbm], "r--", label="y=x")
plt.title(f"Predicted vs. Actual LogP\n{lgbm_model_name}", fontsize=14)
plt.xlabel("Actual LogP", fontsize=12)
plt.ylabel("Predicted LogP", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=10)
plt.axis("equal")
plt.tight_layout()
plt.savefig("scatter_lgbm.png")